# Korduvad närvivõrgud

Eelmises moodulis kasutasime tekstide rikkalikke semantilisi esitusi ja lihtsat lineaarset klassifikaatorit, mis põhines nendele sisenditel. Selline arhitektuur suudab tabada lause sõnade koondatud tähendust, kuid ei arvesta sõnade **järjekorda**, kuna sisendite koondamine eemaldab selle informatsiooni algsest tekstist. Kuna need mudelid ei suuda modelleerida sõnade järjestust, ei ole nad võimelised lahendama keerukamaid või mitmetähenduslikke ülesandeid, nagu teksti genereerimine või küsimustele vastamine.

Tekstijärjestuse tähenduse tabamiseks peame kasutama teistsugust närvivõrgu arhitektuuri, mida nimetatakse **korduvaks närvivõrguks** ehk RNN-iks. RNN-is edastame oma lause läbi võrgu ühe sümboli kaupa, ja võrk genereerib mingi **oleku**, mille edastame võrku uuesti koos järgmise sümboliga.

<img alt="RNN" src="../../../../../translated_images/et/rnn.27f5c29c53d727b5.webp" width="60%"/>

Arvestades sisendjärjestust $X_0,\dots,X_n$, loob RNN järjestuse närvivõrgu plokkidest ja treenib seda järjestust otsast lõpuni tagasileviku meetodil. Iga võrguplokk võtab sisendiks paari $(X_i,S_i)$ ja genereerib tulemuseks $S_{i+1}$. Lõplik olek $S_n$ või väljund $X_n$ suunatakse lineaarsele klassifikaatorile, et toota tulemus. Kõik võrguplokid jagavad samu kaalusid ja neid treenitakse otsast lõpuni ühe tagasileviku käigu abil.

Kuna olekuvektorid $S_0,\dots,S_n$ edastatakse läbi võrgu, suudab see õppida sõnade järjestusevahelisi sõltuvusi. Näiteks, kui sõna *not* ilmub kuskil järjestuses, võib võrk õppida teatud elemente olekuvektoris eitama, mis viib eitamiseni.

> Kuna kõik RNN plokkide kaalud pildil on jagatud, saab sama pilti kujutada ühe plokina (paremal), millel on korduv tagasisideahel, mis edastab võrgu väljundoleku tagasi sisendisse.

Vaatame, kuidas korduvad närvivõrgud aitavad meil klassifitseerida meie uudiste andmekogumit.


In [1]:
import torch
import torchtext
from torchnlp import *
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_size = len(vocab)

Loading dataset...
Building vocab...


## Lihtne RNN klassifikaator

Lihtsa RNN-i puhul on iga korduvüksus lihtne lineaarne võrk, mis võtab sisendvektori ja olekuvektori ühendatud kujul ning toodab uue olekuvektori. PyTorch esindab seda üksust `RNNCell` klassiga, ja selliste rakkude võrku - `RNN` kihina.

RNN klassifikaatori määratlemiseks rakendame esmalt sisendvokaabli dimensioonide vähendamiseks sisestuskihi (embedding layer) ja seejärel lisame sellele RNN kihi:


In [2]:
class RNNClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_class):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.rnn = torch.nn.RNN(embed_dim,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, num_class)

    def forward(self, x):
        batch_size = x.size(0)
        x = self.embedding(x)
        x,h = self.rnn(x)
        return self.fc(x.mean(dim=1))

> **Märkus:** Siin kasutame lihtsuse huvides treenimata sisendkihti, kuid veelgi paremate tulemuste saavutamiseks võiks kasutada eelnevalt treenitud sisendkihti, mis põhineb Word2Vec või GloVe vektoritel, nagu kirjeldatud eelmises osas. Paremaks arusaamiseks võiksite kohandada seda koodi, et see töötaks eelnevalt treenitud vektoritega.

Meie puhul kasutame täiendatud andmete laadijat, nii et iga partii sisaldab sama pikkusega täiendatud järjestusi. RNN-kiht võtab sisendiks järjestuse vektorite tensoritest ja annab kaks väljundit:
* $x$ on RNN-raku väljundite järjestus igal sammul
* $h$ on viimane varjatud olek järjestuse viimase elemendi jaoks

Seejärel rakendame täielikult ühendatud lineaarse klassifikaatori, et saada klasside arv.

> **Märkus:** RNN-e on üsna keeruline treenida, sest kui RNN-rakud lahti rullitakse järjestuse pikkuse ulatuses, on tagasipropagatsioonis osalevate kihtide arv üsna suur. Seetõttu peame valima väikese õppemäära ja treenima võrku suurema andmekogumi peal, et saavutada häid tulemusi. See võib võtta üsna kaua aega, seega on eelistatud GPU kasutamine.


In [3]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=padify, shuffle=True)
net = RNNClassifier(vocab_size,64,32,len(classes)).to(device)
train_epoch(net,train_loader, lr=0.001)

3200: acc=0.3090625
6400: acc=0.38921875
9600: acc=0.4590625
12800: acc=0.511953125
16000: acc=0.5506875
19200: acc=0.57921875
22400: acc=0.6070089285714285
25600: acc=0.6304296875
28800: acc=0.6484027777777778
32000: acc=0.66509375
35200: acc=0.6790056818181818
38400: acc=0.6929166666666666
41600: acc=0.7035817307692308
44800: acc=0.7137276785714286
48000: acc=0.72225
51200: acc=0.73001953125
54400: acc=0.7372794117647059
57600: acc=0.7436631944444444
60800: acc=0.7503947368421052
64000: acc=0.75634375
67200: acc=0.7615773809523809
70400: acc=0.7662642045454545
73600: acc=0.7708423913043478
76800: acc=0.7751822916666666
80000: acc=0.7790625
83200: acc=0.7825
86400: acc=0.7858564814814815
89600: acc=0.7890513392857142
92800: acc=0.7920474137931034
96000: acc=0.7952708333333334
99200: acc=0.7982258064516129
102400: acc=0.80099609375
105600: acc=0.8037594696969697
108800: acc=0.8060569852941176


## Pika- ja lühimälu (LSTM)

Üks klassikaliste korduvate närvivõrkude (RNN) peamisi probleeme on nn **hajuvate gradientide probleem**. Kuna RNN-e treenitakse otsast lõpuni ühe tagasileviku käigus, on neil keeruline viga võrgu esimestesse kihtidesse edasi kanda, mistõttu ei suuda võrk õppida kaugemate tokenite vahelisi seoseid. Üks viis selle probleemi vältimiseks on **eksplitsiitse oleku haldamise** kasutuselevõtt, kasutades nn **väravaid**. Selle tüüpi arhitektuuridest on kaks kõige tuntumat: **Pika- ja lühimälu** (LSTM) ja **Gated Relay Unit** (GRU).

![Pilt, mis näitab näidet pika- ja lühimälu rakust](../../../../../lessons/5-NLP/16-RNN/images/long-short-term-memory-cell.svg)

LSTM-võrk on organiseeritud sarnaselt RNN-iga, kuid seal on kaks olekut, mis edastatakse kihilt kihile: tegelik olek $c$ ja peidetud vektor $h$. Igas üksuses ühendatakse peidetud vektor $h_i$ sisendiga $x_i$, ja need kontrollivad, mis juhtub olekuga $c$ läbi **väravate**. Iga värav on närvivõrk sigmoidse aktivatsiooniga (väljund vahemikus $[0,1]$), mida võib käsitleda kui bitimaski, kui see korrutatakse olekuvektoriga. Järgnevad väravad on olemas (ülaltoodud pildil vasakult paremale):
* **unustamisvärav** võtab peidetud vektori ja määrab, millised komponendid vektorist $c$ tuleb unustada ja millised edasi anda.
* **sisendvärav** võtab osa teavet sisendist ja peidetud vektorist ning lisab selle olekusse.
* **väljundvärav** teisendab oleku läbi mingi lineaarse kihi $\tanh$ aktivatsiooniga, seejärel valib mõned selle komponendid, kasutades peidetud vektorit $h_i$, et toota uus olek $c_{i+1}$.

Olekukomponente $c$ võib käsitleda kui lippe, mida saab sisse ja välja lülitada. Näiteks, kui kohtame järjestuses nime *Alice*, võime eeldada, et see viitab naissoost tegelasele, ja tõsta olekus lippu, mis näitab, et lauses on naissoost nimisõna. Kui kohtame hiljem fraasi *ja Tom*, tõstame lippu, mis näitab, et meil on mitmuse nimisõna. Seega, olekuga manipuleerides saame väidetavalt jälgida lause osade grammatilisi omadusi.

> **Note**: Suurepärane ressurss LSTM-i sisemuse mõistmiseks on Christopher Olah' suurepärane artikkel [Understanding LSTM Networks](https://colah.github.io/posts/2015-08-Understanding-LSTMs/).

Kuigi LSTM-raku sisemine struktuur võib tunduda keeruline, peidab PyTorch selle teostuse `LSTMCell` klassi sisse ja pakub `LSTM` objekti, et esindada tervet LSTM kihti. Seega on LSTM-klassifikaatori teostus üsna sarnane lihtsa RNN-iga, mida me eespool nägime:


In [4]:
class LSTMClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_class):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.embedding.weight.data = torch.randn_like(self.embedding.weight.data)-0.5
        self.rnn = torch.nn.LSTM(embed_dim,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, num_class)

    def forward(self, x):
        batch_size = x.size(0)
        x = self.embedding(x)
        x,(h,c) = self.rnn(x)
        return self.fc(h[-1])

Nüüd treenime oma võrku. Pange tähele, et LSTM-i treenimine on samuti üsna aeglane ja treeningu alguses ei pruugi täpsus märkimisväärselt tõusta. Samuti peate võib-olla katsetama `lr` õppemäära parameetriga, et leida õppemäär, mis tagab mõistliku treeningkiiruse, kuid ei põhjusta mälu raiskamist.


In [5]:
net = LSTMClassifier(vocab_size,64,32,len(classes)).to(device)
train_epoch(net,train_loader, lr=0.001)

3200: acc=0.259375
6400: acc=0.25859375
9600: acc=0.26177083333333334
12800: acc=0.2784375
16000: acc=0.313
19200: acc=0.3528645833333333
22400: acc=0.3965625
25600: acc=0.4385546875
28800: acc=0.4752777777777778
32000: acc=0.505375
35200: acc=0.5326704545454546
38400: acc=0.5557552083333334
41600: acc=0.5760817307692307
44800: acc=0.5954910714285714
48000: acc=0.6118333333333333
51200: acc=0.62681640625
54400: acc=0.6404779411764706
57600: acc=0.6520138888888889
60800: acc=0.662828947368421
64000: acc=0.673546875
67200: acc=0.6831547619047619
70400: acc=0.6917897727272727
73600: acc=0.6997146739130434
76800: acc=0.707109375
80000: acc=0.714075
83200: acc=0.7209134615384616
86400: acc=0.727037037037037
89600: acc=0.7326674107142858
92800: acc=0.7379633620689655
96000: acc=0.7433645833333333
99200: acc=0.7479032258064516
102400: acc=0.752119140625
105600: acc=0.7562405303030303
108800: acc=0.76015625
112000: acc=0.7641339285714286
115200: acc=0.7677777777777778
118400: acc=0.77112331081

(0.03487814127604167, 0.7728)

## Pakitud järjestused

Meie näites pidime täitma kõik minibatch'i järjestused nullvektoritega. Kuigi see põhjustab mõningast mälukasutuse raiskamist, on RNN-ide puhul veelgi olulisem, et täiendavad RNN-rakud luuakse täidetud sisendite jaoks, mis osalevad treeningus, kuid ei kanna endas olulist sisendinfot. Oleks palju parem treenida RNN-i ainult tegeliku järjestuse pikkuse ulatuses.

Selleks on PyTorchis kasutusele võetud spetsiaalne täidetud järjestuste salvestusvorming. Oletame, et meil on sisendiks täidetud minibatch, mis näeb välja selline:
```
[[1,2,3,4,5],
 [6,7,8,0,0],
 [9,0,0,0,0]]
```
Siin tähistab 0 täidetud väärtusi ja sisendjärjestuste tegelik pikkusvektor on `[5,3,1]`.

Selleks, et RNN-i tõhusalt treenida täidetud järjestustega, tahame alustada esimese RNN-rakkude grupi treenimist suure minibatch'iga (`[1,6,9]`), kuid seejärel lõpetada kolmanda järjestuse töötlemine ja jätkata treenimist lühemate minibatch'idega (`[2,7]`, `[3,8]`) ja nii edasi. Seega on pakitud järjestus esitatud ühe vektorina - meie näites `[1,6,9,2,7,3,8,4,5]` ja pikkusvektorina (`[5,3,1]`), mille abil saame hõlpsasti taastada algse täidetud minibatch'i.

Pakitud järjestuse loomiseks saame kasutada funktsiooni `torch.nn.utils.rnn.pack_padded_sequence`. Kõik korduvad kihid, sealhulgas RNN, LSTM ja GRU, toetavad pakitud järjestusi sisendina ja toodavad pakitud väljundi, mida saab dekodeerida funktsiooni `torch.nn.utils.rnn.pad_packed_sequence` abil.

Selleks, et oleks võimalik luua pakitud järjestust, peame võrgule edastama pikkusvektori ja seetõttu vajame minibatch'ide ettevalmistamiseks teistsugust funktsiooni:


In [6]:
def pad_length(b):
    # build vectorized sequence
    v = [encode(x[1]) for x in b]
    # compute max length of a sequence in this minibatch and length sequence itself
    len_seq = list(map(len,v))
    l = max(len_seq)
    return ( # tuple of three tensors - labels, padded features, length sequence
        torch.LongTensor([t[0]-1 for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v]),
        torch.tensor(len_seq)
    )

train_loader_len = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=pad_length, shuffle=True)

Tegelik võrk oleks väga sarnane ülaltoodud `LSTMClassifier`-iga, kuid `forward`-protsess võtab vastu nii täidetud minibatch'i kui ka järjestuste pikkuste vektori. Pärast sisendi teisendamist arvutame pakitud järjestuse, edastame selle LSTM-kihile ja seejärel pakime tulemuse tagasi lahti.

> **Märkus**: Tegelikult me ei kasuta lahtipakitud tulemust `x`, kuna kasutame järgnevates arvutustes peidetud kihtide väljundit. Seega võime selle koodist lahtipakkimise täielikult eemaldada. Põhjus, miks me selle siia lisame, on see, et teil oleks lihtsam koodi muuta, kui peaksite võrgustiku väljundit edaspidistes arvutustes kasutama.


In [7]:
class LSTMPackClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_class):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.embedding.weight.data = torch.randn_like(self.embedding.weight.data)-0.5
        self.rnn = torch.nn.LSTM(embed_dim,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, num_class)

    def forward(self, x, lengths):
        batch_size = x.size(0)
        x = self.embedding(x)
        pad_x = torch.nn.utils.rnn.pack_padded_sequence(x,lengths,batch_first=True,enforce_sorted=False)
        pad_x,(h,c) = self.rnn(pad_x)
        x, _ = torch.nn.utils.rnn.pad_packed_sequence(pad_x,batch_first=True)
        return self.fc(h[-1])

Nüüd alustame treeningut:


In [8]:
net = LSTMPackClassifier(vocab_size,64,32,len(classes)).to(device)
train_epoch_emb(net,train_loader_len, lr=0.001,use_pack_sequence=True)


3200: acc=0.285625
6400: acc=0.33359375
9600: acc=0.3876041666666667
12800: acc=0.44078125
16000: acc=0.4825
19200: acc=0.5235416666666667
22400: acc=0.5559821428571429
25600: acc=0.58609375
28800: acc=0.6116666666666667
32000: acc=0.63340625
35200: acc=0.6525284090909091
38400: acc=0.668515625
41600: acc=0.6822596153846154
44800: acc=0.6948214285714286
48000: acc=0.7052708333333333
51200: acc=0.71521484375
54400: acc=0.7239889705882353
57600: acc=0.7315277777777778
60800: acc=0.7388486842105263
64000: acc=0.74571875
67200: acc=0.7518303571428572
70400: acc=0.7576988636363636
73600: acc=0.7628940217391305
76800: acc=0.7681510416666667
80000: acc=0.7728125
83200: acc=0.7772235576923077
86400: acc=0.7815393518518519
89600: acc=0.7857700892857142
92800: acc=0.7895043103448276
96000: acc=0.7930520833333333
99200: acc=0.7959072580645161
102400: acc=0.798994140625
105600: acc=0.802064393939394
108800: acc=0.8051378676470589
112000: acc=0.8077857142857143
115200: acc=0.8104600694444445
118400

(0.029785829671223958, 0.8138166666666666)

> **Märkus:** Võite olla märganud parameetrit `use_pack_sequence`, mida edastame treeningfunktsioonile. Hetkel nõuab `pack_padded_sequence` funktsioon, et pikkuse järjestuse tensor oleks CPU seadmel, mistõttu peab treeningfunktsioon vältima pikkuse järjestuse andmete GPU-le liigutamist treeningu ajal. Võite vaadata `train_emb` funktsiooni teostust [`torchnlp.py`](../../../../../lessons/5-NLP/16-RNN/torchnlp.py) failis.


## Kahepoolne ja mitmekihiline RNN

Meie näidetes töötasid kõik korduvad võrgud ühes suunas, järjestuse algusest lõpuni. See tundub loomulik, kuna see sarnaneb viisiga, kuidas me loeme ja kuulame kõnet. Kuid paljudel praktilistel juhtudel on meil juhuslik juurdepääs sisendjärjestusele, mistõttu võib olla mõistlik käivitada korduv arvutus mõlemas suunas. Selliseid võrke nimetatakse **kahepoolseteks** RNN-ideks ning neid saab luua, lisades RNN/LSTM/GRU konstruktorile parameetri `bidirectional=True`.

Kahepoolse võrgu puhul vajame kahte varjatud oleku vektorit, üks iga suuna jaoks. PyTorch kodeerib need vektorid üheks kaks korda suurema suurusega vektoriks, mis on üsna mugav, kuna tavaliselt edastatakse saadud varjatud olek täielikult ühendatud lineaarsele kihile. Selle kihi loomisel tuleb lihtsalt arvestada suuruse suurenemisega.

Korduv võrk, olgu see ühe- või kahepoolne, tuvastab teatud mustrid järjestuses ja suudab need salvestada oleku vektorisse või edastada väljundisse. Nagu konvolutsioonivõrkude puhul, saame ehitada esimese kihi peale teise korduva kihi, et tuvastada kõrgema taseme mustreid, mis on loodud esimese kihi poolt tuvastatud madalama taseme mustritest. See viib meid **mitmekihilise RNN-i** mõisteni, mis koosneb kahest või enamast korduvast võrgust, kus eelmise kihi väljund edastatakse järgmisele kihile sisendina.

![Pilt, mis näitab mitmekihilist pika-lühiajalise-mälu RNN-i](../../../../../translated_images/et/multi-layer-lstm.dd975e29bb2a59fe.webp)

*Pilt [sellest suurepärasest postitusest](https://towardsdatascience.com/from-a-lstm-cell-to-a-multilayer-lstm-network-with-pytorch-2899eb5696f3) autorilt Fernando López*

PyTorch muudab selliste võrkude loomise lihtsaks, kuna piisab, kui lisada RNN/LSTM/GRU konstruktorile parameeter `num_layers`, et automaatselt ehitada mitu korduvate kihtide taset. See tähendab ka, et varjatud oleku vektori suurus suureneb proportsionaalselt, ja seda tuleb arvestada korduvate kihtide väljundiga töötamisel.


## RNN-id muude ülesannete jaoks

Selles osas oleme näinud, et RNN-e saab kasutada järjestuste klassifitseerimiseks, kuid tegelikult suudavad need toime tulla paljude teiste ülesannetega, nagu teksti genereerimine, masintõlge ja palju muud. Neid ülesandeid käsitleme järgmises osas.



---

**Lahtiütlus**:  
See dokument on tõlgitud AI tõlketeenuse [Co-op Translator](https://github.com/Azure/co-op-translator) abil. Kuigi püüame tagada täpsust, palume arvestada, et automaatsed tõlked võivad sisaldada vigu või ebatäpsusi. Algne dokument selle algses keeles tuleks pidada autoriteetseks allikaks. Olulise teabe puhul soovitame kasutada professionaalset inimtõlget. Me ei vastuta selle tõlke kasutamisest tulenevate arusaamatuste või valesti tõlgenduste eest.
